In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=a3af371c1b56cd4e2b92fe5a90d305699b2c33dfafb04070b2eb38df572276ed
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql.functions import count, desc, col, max
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/56. Music Recommender system/genre.csv"

'/content/drive/My Drive/Colab Notebooks/56. Music Recommender system/genre.csv'


In [ ]:
spark=SparkSession.builder.appName("lastfm").getOrCreate()

## Loading the Dataset

In [ ]:
file_path='/content/drive/My Drive/Colab Notebooks/56. Music Recommender system/listenings.csv'

In [ ]:
df_listenings=spark.read.format('csv').option('header', True).option('inferSchema', True).load(file_path)
df_listenings.show()

+-----------+-------------+--------------------+---------------+--------------------+
|    user_id|         date|               track|         artist|               album|
+-----------+-------------+--------------------+---------------+--------------------+
|000Silenced|1299680100000|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|1299679920000|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|1299679440000|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|1299679200000|            Acapella|          Kelis|            Acapella|
|000Silenced|1299675660000|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|1297511400000|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|1294498440000|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|1292438340000|               ObZen|      Meshuggah|               ObZen|
|000Silenced|1292437740000|   Yama's Messengers|      

In [ ]:
df_listenings=df_listenings.drop("date")
df_listenings.show()

+-----------+--------------------+---------------+--------------------+
|    user_id|               track|         artist|               album|
+-----------+--------------------+---------------+--------------------+
|000Silenced|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|            Acapella|          Kelis|            Acapella|
|000Silenced|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|               ObZen|      Meshuggah|               ObZen|
|000Silenced|   Yama's Messengers|         Gojira|The Way of All Flesh|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For No...|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For

In [ ]:
df_listenings=df_listenings.na.drop()
df_listenings.show()

+-----------+--------------------+---------------+--------------------+
|    user_id|               track|         artist|               album|
+-----------+--------------------+---------------+--------------------+
|000Silenced|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|            Acapella|          Kelis|            Acapella|
|000Silenced|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|               ObZen|      Meshuggah|               ObZen|
|000Silenced|   Yama's Messengers|         Gojira|The Way of All Flesh|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For No...|
|000Silenced|On the Brink of E...|   Napalm Death|Time Waits For

## Perform the aggregation

In [ ]:
row_numbers=df_listenings.count()
column_numbers=len(df_listenings.columns)
print(row_numbers, column_numbers)

13758905 4


In [ ]:
df_listenings_agg=df_listenings.select('user_id', 'track').groupby('user_id','track').agg(count('*').alias('count')).orderBy('user_id')
df_listenings_agg.show()

+-------+--------------------+-----+
|user_id|               track|count|
+-------+--------------------+-----+
| --Seph| White Winter Hymnal|    3|
| --Seph|         The Funeral|    1|
| --Seph|Hope There's Someone|    1|
| --Seph|         The Painter|    1|
| --Seph|          Je te veux|    1|
| --Seph|            War Pigs|    1|
| --Seph|                 F12|    1|
| --Seph|                Team|    1|
| --Seph|          Nightmares|    1|
| --Seph|               Radio|    1|
| --Seph|   All I Want Is You|    1|
| --Seph|    Little by Little|    2|
| --Seph|        After Nature|    1|
| --Seph|In the Hall of th...|    1|
| --Seph|   Hey There Delilah|    1|
| --Seph|   Let's Call It Off|    1|
| --Seph|               Leloo|    1|
| --Seph|             Pack Up|    1|
| --Seph|           Introitus|    1|
| --Seph|        The Leanover|    1|
+-------+--------------------+-----+
only showing top 20 rows



In [ ]:
row_numbers=df_listenings_agg.count()
column_numbers=len(df_listenings_agg.columns)
print(row_numbers, column_numbers)

9930128 3


In [ ]:
df_listenings_agg=df_listenings_agg.limit(20000)

## Converting string columns into index columns

In [ ]:
indexer = [StringIndexer(inputCol=col, outputCol=col+'_index', handleInvalid='keep').fit(df_listenings_agg) for col in list(set(df_listenings_agg.columns)-set(['count']))]
pipeline = Pipeline(stages=indexer)

data=pipeline.fit(df_listenings_agg).transform(df_listenings_agg)
data.show

<bound method DataFrame.show of DataFrame[user_id: string, track: string, count: bigint, user_id_index: double, track_index: double]>

In [ ]:
data=data.select('user_id_index','track_index', 'count').orderBy('user_id_index')

In [ ]:
data.show()

+-------------+-----------+-----+
|user_id_index|track_index|count|
+-------------+-----------+-----+
|          0.0|    10628.0|    1|
|          0.0|     3338.0|    1|
|          0.0|    12168.0|    1|
|          0.0|    11626.0|    2|
|          0.0|    10094.0|    4|
|          0.0|      427.0|    1|
|          0.0|    16878.0|    1|
|          0.0|    11722.0|    1|
|          0.0|    15074.0|    1|
|          0.0|     1359.0|    1|
|          0.0|     5874.0|    1|
|          0.0|    11184.0|    1|
|          0.0|     2372.0|    2|
|          0.0|    14316.0|    1|
|          0.0|     5346.0|    1|
|          0.0|    11194.0|    1|
|          0.0|     2241.0|    1|
|          0.0|     2864.0|    1|
|          0.0|     2663.0|    4|
|          0.0|     6064.0|    1|
+-------------+-----------+-----+
only showing top 20 rows



## Creating the ALS model for our recommendation system

In [ ]:
(training, test)=data.randomSplit([0.5, 0.5])

In [ ]:
USERID='user_id_index'
TRACK='track_index'
COUNT='count'

als=ALS(maxIter=5, regParam=0.01, userCol=USERID, itemCol=TRACK, ratingCol=COUNT)
model=als.fit(training)

predictions= model.transform(test)

In [ ]:
recs=model.recommendForAllUsers(10)
recs.show()

+-------------+--------------------+
|user_id_index|     recommendations|
+-------------+--------------------+
|            0|[{233, 9.736242},...|
|            1|[{11940, 37.14650...|
|            2|[{9321, 8.0973}, ...|
|            3|[{8819, 14.668208...|
|            4|[{11940, 17.0771}...|
|            5|[{11940, 16.26232...|
|            6|[{11940, 25.52384...|
|            7|[{15450, 8.89087}...|
|            8|[{11940, 24.07358...|
|            9|[{9498, 9.972982}...|
|           10|[{1689, 8.663973}...|
|           11|[{11940, 16.67781...|
|           12|[{349, 7.84948}, ...|
|           13|[{2245, 7.8181214...|
|           14|[{1694, 6.0334105...|
|           15|[{1689, 6.4752994...|
|           16|[{8819, 10.633419...|
|           17|[{3345, 10.688244...|
|           18|[{11940, 26.93067...|
|           19|[{9498, 10.172409...|
+-------------+--------------------+
only showing top 20 rows



In [ ]:
recs.take(1)

[Row(user_id_index=0, recommendations=[Row(track_index=233, rating=9.736242294311523), Row(track_index=101, rating=9.591691970825195), Row(track_index=12844, rating=8.590802192687988), Row(track_index=8702, rating=8.590802192687988), Row(track_index=6515, rating=8.590802192687988), Row(track_index=4220, rating=8.590802192687988), Row(track_index=2128, rating=8.590802192687988), Row(track_index=3508, rating=8.018081665039062), Row(track_index=14798, rating=7.445361614227295), Row(track_index=14666, rating=6.29992151260376)])]